In [1]:
from dotenv import load_dotenv
load_dotenv()
from langchain.callbacks import get_openai_callback
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

In [2]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")

    # Throw error in case of receiving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
        for item in field:
            if item[0].isnumeric():
                raise ValueError("The word can not start with numbers!")
        return field

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [3]:
from langchain.prompts import PromptTemplate

template = """
Offer a list of suggestions to substitue the specified target_word based the presented context.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt.format_prompt(
			target_word="behaviour",
			context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."
)

In [7]:
model = OpenAI(model_name='gpt-3.5-turbo', temperature=0.0)

with get_openai_callback() as cb:
    output = model(model_input.to_string())
    print(output)
    print(cb)

with get_openai_callback() as cb:
    print(parser.parse(output))
    print(cb)

/workspaces/ActiveLoop-Course/env/lib/python3.10/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/workspaces/ActiveLoop-Course/env/lib/python3.10/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


{
    "words": ["conduct", "actions", "manners", "demeanor", "attitude"]
}
Tokens Used: 247
	Prompt Tokens: 222
	Completion Tokens: 25
Successful Requests: 1
Total Cost (USD): $0.00038300000000000004
words=['conduct', 'actions', 'manners', 'demeanor', 'attitude']
Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [8]:
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")
    
    @validator('words')
    def not_start_with_number(cls, field):
      for item in field:
        if item[0].isnumeric():
          raise ValueError("The word can not start with numbers!")
      return field
    
    @validator('reasons')
    def end_with_dot(cls, field):
      for idx, item in enumerate( field ):
        if item[-1] != ".":
          field[idx] += "."
      return field
    
with get_openai_callback() as cb:
    output = model(model_input.to_string())
    print(output)
    print(cb)

with get_openai_callback() as cb:
    print(parser.parse(output))
    print(cb)

{
    "words": ["conduct", "actions", "demeanor", "attitude", "conduct"]
}
Tokens Used: 246
	Prompt Tokens: 222
	Completion Tokens: 24
Successful Requests: 1
Total Cost (USD): $0.00038100000000000005
words=['conduct', 'actions', 'demeanor', 'attitude', 'conduct']
Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0
